In [1]:
#import os
from sympy import *
import numpy as np
from tabulate import tabulate
from scipy import signal
import matplotlib.pyplot as plt
import pandas as pd
from SymNode import SymMNA
from IPython.display import display, Markdown, Math, Latex
init_printing()

In [2]:
!pip install ollama

In [3]:
net_list = '''
I1 1 0 1 
R1 1 0 0.05
L1 1 2 0.025
C1 2 0 0.25
'''

In [4]:
import sympy as sp
from sympy import Matrix, Eq, latex, solve
from tabulate import tabulate

# Define 's' if it's not already defined
s = sp.Symbol('s')

# Run Modified Nodal Analysis (MNA)
report, network_df, i_unk_df, A, X, Z = SymMNA.smna(net_list)

# Convert X and Z into matrix form for better LaTeX formatting
Xp = Matrix(X)
Zp = Matrix(Z)

# Generate LaTeX equations for the system of equations
temp_formular = ''
for i in range(len(X)):
    temp_formular += '${:s}$<br>'.format(latex(Eq((A * Xp)[i], Zp[i])))

# Convert matrices into SymPy expressions
X = Matrix(X)
Z = Matrix(Z)

# Formulate the system of equations symbolically
NE_sym = Eq(A * X, Z)
U_sym = solve(NE_sym, X)

# Generate LaTeX expressions for the solved unknowns
temp_solve = ''
for var, sol in U_sym.items():
    temp_solve += '${:s} = {:s}$<br>'.format(latex(var), latex(sol))

# Retrieve component values from the network
element_values = SymMNA.get_part_values(network_df)
element_values_list = []
for k, v in element_values.items():
    element_values_list.append([f"{k} = {v}"])
    print(f"{k} = {v}")

# Display the component values
for k, v in element_values.items():
    print(f"{k} = {v}")

# Substitute known component values into the system
NE = NE_sym.subs(element_values)

# Debug: Check if 's' exists in the equation
print("Symbols in NE:", NE.free_symbols)

# Perform DC analysis by setting s = 0 (only if 's' is present)
if s in NE.free_symbols:
    NE_dc = NE.subs({s: 0})
else:
    NE_dc = NE  # No need to substitute if 's' isn't present

# Generate LaTeX expressions for the DC analysis equations
temp_DC = ''
for i in range(NE_dc.lhs.shape[0]):
    temp_DC += '${:s} = {:s}$<br>'.format(latex(NE_dc.rhs[i]), latex(NE_dc.lhs[i]))

# Solve for DC operating point
U_dc = solve(NE_dc, X)

# Prepare results for tabulation
table_header = ['Unknown', 'Magnitude']
table_row = [[str(name), float(value)] for name, value in U_dc.items()]

# Display the results in a formatted table
result_table = tabulate(table_row, headers=table_header, colalign=('left', 'decimal'), 
                        tablefmt="simple", floatfmt=('.5s', '.6f'))

# Print the formatted result table
print(result_table)


I1 = 1.0
R1 = 0.05
L1 = 0.025
C1 = 0.25
I1 = 1.0
R1 = 0.05
L1 = 0.025
C1 = 0.25
Symbols in NE: {v1, s, I_L1, v2}
Unknown      Magnitude
---------  -----------
I_L1          0.000000
v1           -0.050000
v2           -0.050000


In [5]:
from sympy import symbols, Eq, pprint

# deepseek r1:1.5B

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
    
context_str = f'''

Context for Circuit Analysis LLM

This document explains the circuit analysis process by defining components, deriving equations, 
performing DC analysis, and presenting results. The structured approach ensures accurate analysis and interpretation of circuit behavior 
and explain and show all equaltion.

1. Circuit Equations
The mathematical formulations used in the analysis are:

{temp_formular}
These equations are derived from fundamental circuit laws, including Kirchhoff’s Laws and Ohm’s Law.

2. Circuit Analysis Report
A detailed explanation of the circuit, its components, and how they interact:


{report}
This section provides insights into circuit topology, applied sources, and expected behavior.

3. Solution Approach & Computation Steps
The methodology and step-by-step solution for solving the circuit equations:


{temp_solve}
This may include algebraic manipulations, matrix-based solutions, or numerical methods.

4. DC Analysis and Component Values
The list of circuit element values used in the DC analysis:


{element_values_list}
The computed results from the DC analysis:

{temp_DC}
5. Final Computed Results Table
A structured table summarizing voltages, currents, and power calculations:

{result_table}
This table presents the final numerical results for easy interpretation.
'''
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="deepseek-r1:1.5b")

chain = prompt | model
response = chain.invoke({"question":"can you analyze this circuit? and expand all step by step and give all equation","context": context_str})
display(Math(response))

<IPython.core.display.Math object>

# gemma 3

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
    
context_str = f'''

Context for Circuit Analysis LLM

This document explains the circuit analysis process by defining components, deriving equations, 
performing DC analysis, and presenting results. The structured approach ensures accurate analysis and interpretation of circuit behavior 
and explain and show all equaltion.

1. Circuit Equations
The mathematical formulations used in the analysis are:

{temp_formular}
These equations are derived from fundamental circuit laws, including Kirchhoff’s Laws and Ohm’s Law.

2. Circuit Analysis Report
A detailed explanation of the circuit, its components, and how they interact:


{report}
This section provides insights into circuit topology, applied sources, and expected behavior.

3. Solution Approach & Computation Steps
The methodology and step-by-step solution for solving the circuit equations:


{temp_solve}
This may include algebraic manipulations, matrix-based solutions, or numerical methods.

4. DC Analysis and Component Values
The list of circuit element values used in the DC analysis:


{element_values_list}
The computed results from the DC analysis:

{temp_DC}
5. Final Computed Results Table
A structured table summarizing voltages, currents, and power calculations:

{result_table}
This table presents the final numerical results for easy interpretation.
'''
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="gemma3")

chain = prompt | model
response = chain.invoke({"question":"can you analyze this circuit? and expand all step by step and give all equation","context": context_str})
display(Math(response))

<IPython.core.display.Math object>